# Diffusion 進階技術 (Advanced Diffusion Models)

本 notebook 對應李宏毅老師 2025 Spring ML HW10，探討 Diffusion 模型的進階技術。

## 學習目標
1. 理解 DDIM 採樣加速
2. 學習 Classifier-Free Guidance (CFG)
3. 了解 ControlNet 的條件控制
4. 探索 Latent Diffusion (Stable Diffusion) 架構

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1. DDPM vs DDIM

### 1.1 DDPM 採樣（原始方法）
- 需要 1000 步的迭代
- 每步都有隨機噪音

### 1.2 DDIM 採樣（加速方法）
- 可以用 50-100 步達到相似品質
- 確定性採樣（無隨機噪音）

In [ ]:
class DDIMSampler:
    """
    DDIM (Denoising Diffusion Implicit Models) 採樣器
    """
    def __init__(self, num_train_timesteps=1000, beta_start=0.0001, beta_end=0.02):
        self.num_train_timesteps = num_train_timesteps
        
        # Beta schedule
        betas = torch.linspace(beta_start, beta_end, num_train_timesteps)
        alphas = 1.0 - betas
        self.alphas_cumprod = torch.cumprod(alphas, dim=0)
    
    def set_timesteps(self, num_inference_steps):
        """設定採樣步數（比訓練步數少很多）"""
        self.num_inference_steps = num_inference_steps
        step_ratio = self.num_train_timesteps // num_inference_steps
        self.timesteps = torch.arange(0, num_inference_steps) * step_ratio
        self.timesteps = torch.flip(self.timesteps, [0])  # 從大到小
    
    def step(self, model_output, timestep, sample, eta=0.0):
        """
        單步 DDIM 採樣
        
        eta=0: 完全確定性
        eta=1: 等同 DDPM
        """
        prev_timestep = timestep - self.num_train_timesteps // self.num_inference_steps
        
        alpha_prod_t = self.alphas_cumprod[timestep]
        alpha_prod_t_prev = self.alphas_cumprod[prev_timestep] if prev_timestep >= 0 else torch.tensor(1.0)
        
        # 預測 x0
        pred_x0 = (sample - torch.sqrt(1 - alpha_prod_t) * model_output) / torch.sqrt(alpha_prod_t)
        
        # 計算「方向」
        sigma = eta * torch.sqrt((1 - alpha_prod_t_prev) / (1 - alpha_prod_t)) * \
                torch.sqrt(1 - alpha_prod_t / alpha_prod_t_prev)
        
        # 計算 x_{t-1}
        dir_xt = torch.sqrt(1 - alpha_prod_t_prev - sigma**2) * model_output
        prev_sample = torch.sqrt(alpha_prod_t_prev) * pred_x0 + dir_xt
        
        if eta > 0:
            noise = torch.randn_like(sample)
            prev_sample = prev_sample + sigma * noise
        
        return prev_sample

print("DDIM Sampler 已實作")
print("優勢：可將 1000 步降到 50 步，大幅加速生成")

## 2. Classifier-Free Guidance (CFG)

In [ ]:
def classifier_free_guidance(model, x_t, t, condition, guidance_scale=7.5):
    """
    Classifier-Free Guidance
    
    在訓練時，隨機以 p_uncond 機率丟棄條件（用空條件替代）
    在推理時，結合有條件和無條件的預測
    
    公式：
    ε_guided = ε_uncond + guidance_scale * (ε_cond - ε_uncond)
    """
    # 無條件預測
    noise_pred_uncond = model(x_t, t, condition=None)
    
    # 有條件預測
    noise_pred_cond = model(x_t, t, condition=condition)
    
    # CFG 公式
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_cond - noise_pred_uncond)
    
    return noise_pred

print("""CFG 說明：
- guidance_scale = 1.0: 無引導
- guidance_scale = 7.5: 標準值（Stable Diffusion 常用）
- guidance_scale > 10: 更強的條件遵循，但可能失真""")

## 3. Latent Diffusion (Stable Diffusion 架構)

In [ ]:
print("""
┌─────────────────────────────────────────────────────────────────────────┐
│                    Latent Diffusion 架構                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│   Image (512x512x3)    Latent (64x64x4)    Image (512x512x3)            │
│         │                    │                    ▲                     │
│         ▼                    ▼                    │                     │
│   ┌──────────┐        ┌──────────────┐     ┌──────────┐                │
│   │  VAE     │        │   U-Net      │     │  VAE     │                │
│   │ Encoder  │   →    │  Diffusion   │  →  │ Decoder  │                │
│   └──────────┘        │   Process    │     └──────────┘                │
│                       └──────────────┘                                  │
│                              ▲                                          │
│                              │                                          │
│                    ┌─────────────────┐                                  │
│                    │  Text Encoder   │                                  │
│                    │    (CLIP)       │                                  │
│                    └─────────────────┘                                  │
│                              ▲                                          │
│                              │                                          │
│                      "a photo of cat"                                  │
│                                                                         │
│  優勢：                                                                  │
│  1. 在低維 latent 空間操作，計算效率高                                    │
│  2. VAE 壓縮 8 倍，大幅減少計算量                                         │
│  3. 可以處理高解析度圖像                                                  │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
""")

## 4. 練習題

### 練習 1：比較不同 CFG 值的效果

In [ ]:
# 練習 1：實驗不同 guidance_scale 的效果
def compare_guidance_scales(model, prompt, scales=[1.0, 3.0, 7.5, 12.0]):
    """
    TODO: 對比不同 guidance_scale 的生成結果
    
    觀察：
    - scale 太低：圖像可能與 prompt 不相關
    - scale 適中：最佳平衡
    - scale 太高：過度飽和、失真
    """
    pass

print("練習 1：實作 compare_guidance_scales 函數")

## 5. 總結

| 技術 | 用途 |
|------|------|
| DDIM | 加速採樣（1000步→50步） |
| CFG | 提高生成品質和條件遵循 |
| Latent Diffusion | 在壓縮空間操作，高效處理高解析度 |
| ControlNet | 添加額外條件控制（邊緣、姿態等） |

In [ ]:
print("Diffusion 進階技術 - 完成！")